In [ ]:
# implementing Autogen - multi agent conversation
!pip install pyautogen==0.2.3

In [ ]:
from google.colab import userdata
userdata.get('var1')

In [ ]:
%env OAI_CONFIG_LIST=[{"model": "gpt-4","api_key": 'var1'}]

env: OAI_CONFIG_LIST=[{"model": "gpt-4","api_key": 'var1'}]


In [ ]:
# Setting OAI_Config_List
import autogen

api_keys = ['var1']
base_urls = None  # You can specify API base URLs if needed. eg: localhost:8000
api_type = "openai"  # Type of API, e.g., "openai" or "aoai".
api_version = None  # Specify API version if needed.

config_list = autogen.get_config_list(api_keys, base_urls=base_urls, api_type=api_type, api_version=api_version)

print(config_list)

[{'api_key': 'var1', 'api_type': 'openai'}]


#### Let’s initialize both the agents with our LLM and then ask it to solve a math problem:

In [ ]:
from autogen import OpenAIWrapper
import pprint
client = OpenAIWrapper(config_list=config_list)
response = client.create(messages=[{"role": "user", "content": """Find $a + b + c$, given that $x+y \\neq -1$ and
                                        ax + by + c & = x + 7,\\
                                        a + bx + cy & = 2x + 6y,\\
                                        ay + b + cx & = 4x + y.
"""}])
pprint.pprint(client.extract_text_or_completion_object(response))

['First, note that $x + y = r$ for some nonnegative $r$. Then, we can rewrite '
 'the given equations as\n'
 '\\begin{align*}\n'
 'ax + by + c &= 1*x + 7y + 0 \\\\\n'
 'a + bx + cy &= 0*x + 2y + 1 \\\\\n'
 'ay + b + cx &= 4*x + 1y + 0.\n'
 '\\end{align*}\n'
 'By comparing coefficients, we get\n'
 '\\begin{align*}\n'
 'a &= 1, \\\\\n'
 'b &= 2, \\\\\n'
 'c &= 1.\n'
 '\\end{align*}\n'
 'Therefore, $a + b + c = 1 + 2 + 1 = \\boxed{4}$.']


In [ ]:
from autogen import OpenAIWrapper
client = OpenAIWrapper(config_list=config_list)
response = client.create(messages=[{"role": "user", "content": "2+2="}]) # model = "gpt-4"
print(client.extract_text_or_completion_object(response)) #answer = 4

In [ ]:
# using Mathchat now
!pip install "pyautogen[mathchat]<0.2"

In [ ]:
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

In [ ]:
math_problem = "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
mathproxyagent.initiate_chat(assistant, problem=math_problem)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continu

KeyboardInterrupt: 